* Dataset: https://www.kaggle.com/faressayah/audiobook-app-data

* Predicting a metrics to make sure what types of customer will come back again.

In [1]:
#lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [4]:
raw_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NNN/audiobook_data_2.csv')

In [5]:
raw_data.head()

,Unnamed: 0,Book_length(mins)_overall,Book_length(mins)_avg,Price_overall,Price_avg,Review,Review10/10,Completion,Minutes_listened,Support_Request,Last_Visited_mins_Purchase_date,Target
0,994,1620.0,1620,19.73,19.73,1,10.00,0.99,1603.8,5,92,0
1,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
2,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
3,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
4,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0


#### Preprocessing

In [11]:
data = raw_data.copy()
data.columns = [''] * len(data.columns)

In [12]:
data.head()

,,,,,,,,,,,,
0,994,1620.0,1620,19.73,19.73,1,10.00,0.99,1603.8,5,92,0
1,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
2,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
3,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
4,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0


In [13]:
data.shape

(14084, 12)

In [14]:
raw_data.shape

(14084, 12)

In [19]:
#creating Inputs & Targets
unscaled_inputs = data.iloc[:,1:-1]
targets = data.iloc[:,-1]

In [20]:
unscaled_inputs.shape , targets.shape

((14084, 10), (14084,))

In [24]:
### BALANCE THE DATASETS ###
targets.value_counts()

0    11847
1     2237
Name: , dtype: int64

In [25]:
#converting to np array
unscaled_inputs_np = np.array(unscaled_inputs)
targets_np = np.array(targets)

In [30]:
int(np.sum(targets_np)) #total 1

2237

In [31]:
len(targets_np) - int(np.sum(targets_np))  #total 0

11847

In [34]:
num_one_targets = int(np.sum(targets_np))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_np.shape[0]):
  if targets_np[i] == 0:
    zero_targets_counter +=1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)

unscaled_inputs_balance = np.delete(unscaled_inputs_np, indices_to_remove,axis=0)
targets_balance = np.delete(targets_np,indices_to_remove,axis=0)

In [35]:
unscaled_inputs_balance.shape, targets_balance.shape

((4474, 10), (4474,))

In [36]:
#
int(np.sum(targets_balance)) #total 1

2237

In [37]:
#
len(targets_balance) - int(np.sum(targets_balance))  #total 0

2237

In [38]:
#we got balanced targets and inputs

#### Standardize & Shuffle

In [39]:
from sklearn.preprocessing import scale

In [40]:
scaled_inputs_balance = scale(unscaled_inputs_balance)

In [41]:
### Shuffle
shuffled_indices = np.arange(scaled_inputs_balance.shape[0])
np.random.shuffle(shuffled_indices)

shuffle_inputs = scaled_inputs_balance[shuffled_indices]
shuffle_target = targets_balance[shuffled_indices]

In [42]:
scaled_inputs_balance[:5]

array([[ 2.10533868e-01, -1.88885170e-01,  1.97823887e+00,
         1.43819814e+00,  2.10780037e+00,  1.49110879e+00,
         4.24286158e+00,  4.80955413e+00,  1.18382842e+01,
         9.41504310e-02],
       [ 1.27894497e+00,  4.16467444e-01, -3.90824747e-01,
        -5.08367103e-01, -4.74428231e-01,  1.42655343e-03,
        -4.26477846e-01, -4.15699224e-01, -2.01834807e-01,
        -8.02558517e-01],
       [ 1.27894497e+00,  4.16467444e-01, -3.90824747e-01,
        -5.08367103e-01, -4.74428231e-01,  1.42655343e-03,
        -4.26477846e-01, -4.15699224e-01, -2.01834807e-01,
         2.97921400e+00],
       [ 2.10533868e-01, -1.88885170e-01, -2.87178214e-01,
        -4.23204874e-01, -4.74428231e-01,  1.42655343e-03,
         1.55445403e+00,  1.80107493e+00,  2.20618899e+00,
         4.54783377e-01],
       [ 1.27894497e+00,  4.16467444e-01, -3.90824747e-01,
        -5.08367103e-01, -4.74428231e-01,  1.42655343e-03,
         6.11153137e-01,  1.13252399e+00, -2.01834807e-01,
         2.

In [43]:
shuffle_inputs[:5]

array([[-6.44195015e-01, -6.73167261e-01, -3.90824747e-01,
        -5.08367103e-01, -4.74428231e-01,  1.42655343e-03,
        -4.26477846e-01, -4.15699224e-01, -2.01834807e-01,
        -2.66482515e-01],
       [ 2.10533868e-01, -1.88885170e-01,  4.84391317e-02,
        -1.47441464e-01, -4.74428231e-01,  1.42655343e-03,
        -4.26477846e-01, -4.15699224e-01, -2.01834807e-01,
         6.88707451e-01],
       [-8.57877235e-01, -7.94237784e-01,  4.84391317e-02,
        -1.47441464e-01, -4.74428231e-01,  1.42655343e-03,
         4.22492958e-01,  2.17664820e-01, -2.01834807e-01,
        -7.73318008e-01],
       [-6.44195015e-01, -6.73167261e-01,  3.98863125e-01,
         1.40487978e-01,  2.10780037e+00, -1.24225311e+00,
         8.46978360e-01,  6.29351448e-01,  2.20618899e+00,
        -6.36862298e-01],
       [ 2.10533868e-01, -1.88885170e-01,  3.98863125e-01,
         1.40487978e-01, -4.74428231e-01,  1.42655343e-03,
        -4.26477846e-01, -4.15699224e-01, -2.01834807e-01,
        -7.

In [45]:
#### TRAIN TEST VALID
samples_count = shuffle_inputs.shape[0]

train_samples_count = int(0.8* samples_count)
valid_samples_count = int(0.1 *samples_count)
test_samples_count = samples_count - train_samples_count - valid_samples_count

In [52]:
train_data = shuffle_inputs[:train_samples_count]
train_target = shuffle_target[:train_samples_count]

#validation
valid_data = shuffle_inputs[train_samples_count:train_samples_count+valid_samples_count]
valid_target = shuffle_target[train_samples_count:train_samples_count+valid_samples_count]

#Test
test_data = shuffle_inputs[train_samples_count+valid_samples_count:]
test_target = shuffle_target[train_samples_count+valid_samples_count:]

In [53]:
train_data.shape, train_target.shape

((3579, 10), (3579,))

In [54]:
valid_data.shape, valid_target.shape

((447, 10), (447,))

In [55]:
test_data.shape,test_target.shape

((448, 10), (448,))

In [56]:
#saving .npz
np.savez('train_data',inputs= train_data, targets= train_target)
np.savez('valid_data',inputs= valid_data, targets= valid_data)
np.savez('test_data',inputs= test_data, targets= test_data)